<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/05-%E7%89%B9%E5%BE%81%E4%BA%A4%E4%BA%92/07_DeepFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepFM

- wide侧：稀疏输入
- wide侧：field进行FM交互然后输入；
- deep侧：稀疏emb两两进行FM交互，然后输入。


In [ ]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D

## 0.准备工作

In [ ]:
seq = fc.sequence_categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.sequence_categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_wide_col = fc.embedding_column(seq, dimension=1)
target_wide_col = fc.embedding_column(target, dimension=1)
wide_columns = [seq_wide_col, target_wide_col]

seq_v_col = fc.embedding_column(seq, dimension=8)
target_v_col = fc.embedding_column(target, dimension=8)
deep_columns = [seq_v_col, target_v_col]

features={
  "seq": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [2, 0], [2, 1]],
      values=[1100, 1101, 1103, 1102, 1101, 1100, 1103],
      dense_shape=[3, 3]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),
}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1100, 1101, 1103],
       [1102, 1101,    0],
       [1100, 1103,    0]], dtype=int32)>

## 2.wide部分

In [ ]:
wide_input_layer = SequenceFeatures(wide_columns, name='wide_input_layer')
wide_part_dict, wide_len_dict = wide_input_layer(features)
wide_part_dict

{'seq_embedding': <tf.Tensor: shape=(3, 3, 1), dtype=float32, numpy=
 array([[[-1.2562095 ],
         [-0.63669527],
         [ 0.52460396]],
 
        [[-0.4415771 ],
         [-0.63669527],
         [ 0.        ]],
 
        [[-1.2562095 ],
         [ 0.52460396],
         [ 0.        ]]], dtype=float32)>,
 'target_embedding': <tf.Tensor: shape=(3, 1, 1), dtype=float32, numpy=
 array([[[ 0.81646895]],
 
        [[-0.27553457]],
 
        [[-1.1094153 ]]], dtype=float32)>}

In [ ]:
wide_emb = tf.concat(list(wide_part_dict.values()), axis=1)
wide_out = tf.reduce_sum(wide_emb, axis=1)
wide_out

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.55183184],
       [-1.353807  ],
       [-1.8410208 ]], dtype=float32)>

## wide field交叉

In [ ]:
hidden_net = [32, 16]
dnn = [Dense(units) for units in hidden_net]
output = Dense(1)
dnn,output

([<keras.layers.core.dense.Dense at 0x7f40ae3a80d0>,
 <keras.layers.core.dense.Dense at 0x7f40ae3a8150>)

In [ ]:
sequence_feature_layer = SequenceFeatures(deep_columns, name='sequence_features_input_layer')
seq_emb_dict, seq_len_dict = sequence_feature_layer(features)
seq_emb_dict.keys()

dict_keys(['seq_embedding', 'target_embedding'])

In [ ]:
feature_emb = tf.concat(list(seq_emb_dict.values()), axis=1)

In [ ]:
square_sum = tf.square(tf.reduce_sum(feature_emb, axis=1, keepdims=True))  # (batch_size, 1, embed_dim)
sum_square = tf.reduce_sum(tf.square(feature_emb), axis=1, keepdims=True)  # (batch_size, 1, embed_dim)
order_2nd = 0.5 * tf.reduce_sum(square_sum - sum_square, axis=2)  # (batch_size, 1)
order_2nd

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[ 0.67036307],
       [-0.19438568],
       [ 1.0304146 ]], dtype=float32)>

## 2.Deep部分

- 省略两两units交互

In [ ]:
feature_emb_list = [tf.reduce_sum(emb, axis=1) for emb in seq_emb_dict.values()]
feature_emb_list

[<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
 array([[-0.49617663, -0.29706725, -0.9212994 , -0.6201912 , -0.6959002 ,
         -0.743051  ,  0.35907674, -0.6889695 ],
        [-0.0561493 , -0.32312697, -0.34185457, -0.09206233, -0.6287876 ,
         -0.11313982, -0.38679832, -0.34011817],
        [-0.29345843, -0.15343736, -0.71839345, -0.49594462, -0.09418745,
         -0.7388194 ,  0.51937824, -0.75779957]], dtype=float32)>,
 <tf.Tensor: shape=(3, 8), dtype=float32, numpy=
 array([[ 0.41675764,  0.2765483 ,  0.03963033, -0.6217215 ,  0.3022967 ,
         -0.4067798 , -0.23012994, -0.18241058],
        [-0.2553696 ,  0.13021663,  0.22282931,  0.04598617,  0.36279324,
         -0.01382723, -0.45736542,  0.24040934],
        [-0.2442582 , -0.5225778 ,  0.04418829, -0.28923795,  0.2690728 ,
         -0.31855312, -0.3698499 , -0.642562  ]], dtype=float32)>]

In [ ]:
deep_input = tf.concat(feature_emb_list, axis=-1)
deep_input

<tf.Tensor: shape=(3, 16), dtype=float32, numpy=
array([[-0.49617663, -0.29706725, -0.9212994 , -0.6201912 , -0.6959002 ,
        -0.743051  ,  0.35907674, -0.6889695 ,  0.41675764,  0.2765483 ,
         0.03963033, -0.6217215 ,  0.3022967 , -0.4067798 , -0.23012994,
        -0.18241058],
       [-0.0561493 , -0.32312697, -0.34185457, -0.09206233, -0.6287876 ,
        -0.11313982, -0.38679832, -0.34011817, -0.2553696 ,  0.13021663,
         0.22282931,  0.04598617,  0.36279324, -0.01382723, -0.45736542,
         0.24040934],
       [-0.29345843, -0.15343736, -0.71839345, -0.49594462, -0.09418745,
        -0.7388194 ,  0.51937824, -0.75779957, -0.2442582 , -0.5225778 ,
         0.04418829, -0.28923795,  0.2690728 , -0.31855312, -0.3698499 ,
        -0.642562  ]], dtype=float32)>

In [ ]:
for nn in dnn:
  deep_input = nn(deep_input)
outputs = output(deep_input)

In [ ]:
outputs

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.322592  ],
       [0.02042222],
       [0.01225004]], dtype=float32)>

In [ ]:
tf.nn.sigmoid(wide_out + order_2nd + outputs)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.6085266 ],
       [0.17832011],
       [0.31037724]], dtype=float32)>